In [31]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"] = os.getenv("GROK_API_KEY")
model = init_chat_model("groq:qwen/qwen3-32b")
model.invoke("Give me the all description about Avenger movie")

AIMessage(content='<think>\nOkay, the user wants a complete description of the Avengers movie. Let me start by recalling the main Avengers movies. The first one is "Avengers: Assemble" (2012), right? It\'s the first team-up after the individual Marvel movies. I need to mention the plot where Loki tries to take over New York, and the Avengers have to stop him. Key characters like Iron Man, Captain America, Thor, Hulk, Black Widow, and Hawkeye.\n\nThen there\'s "Avengers: Age of Ultron" (2015). Here, Tony Stark creates Ultron, which turns against them. Wanda and Vision join the team. The conflict with Ultron and the final battle in Sokovia. Also, Vision\'s creation and the post-credits scene with Thanos.\n\nNext is "Avengers: Infinity War" (2018). Thanos collects the Infinity Stones to balance the universe. The Battle of Titan where Thanos kills half the population. The snap that leads to the snap in Avengers 4. Important to note the split between the Avengers and the time heist setup.\n

In [32]:
from pydantic import BaseModel,Field
class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int= Field(description="The year the movie was released")
    director:str= Field(description="The director of the movie")
    rating:float= Field(description="The movie rating of 10")

In [33]:
model_with_structure = model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000026C1C469E50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026C1C46A850>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'The year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movie rating of 10', 'type': 'number'}}, 'required': ['title', 'year', 'direct

In [34]:
response = model.stream("Provide the detail about the movie Inception")
for r in response:
    print(r.text,end="", flush=True)

<think>
Okay, so I need to find detailed information about the movie Inception. Let me start by recalling what I know. Inception is a 2010 movie directed by Christopher Nolan. The main actors are Leonardo DiCaprio and some others like Joseph Gordon-Levitt and Tom Hardy. The plot involves dreams and reality, right? The title "Inception" probably refers to inserting an idea into someone's mind through their dreams.

First, I should check the director and main cast to confirm. Christopher Nolan is known for complex narratives, like in Memento and The Dark Knight. The cast includes DiCaprio as Dom Cobb, the protagonist. Then there's Arthur (Joseph Gordon-Levitt), Eames (Tom Hardy), and Ariadne (Elliot Page). The villain might be Mal (Marion Cotillard), Cobb's wife. The plot centers around Cobb's team trying to plant an idea into a target's subconscious. They use a device called the PASIV, which allows them to enter people's dreams. The concept is similar to "The Matrix," where reality is m

In [35]:
#output with the format
model_with_structure.invoke("Provide the detail about the movie Inception")

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

### Message  output alogside  Parsed structure


In [36]:
from pydantic import BaseModel,Field
class Movie(BaseModel):
    title:str =Field(...,description="The title of the movie")
    year:int = Field(..., description="The release year of the movie")
    rating:float = Field(..., description="The total rating of the movie")

model_with_structure = model.with_structured_output(Movie, include_raw=True)
getresponse = model_with_structure.invoke("Provide the details about the movie Inception")
getresponse

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Inception. Let me check what functions I have available. There\'s a Movie function that requires title, year, and rating. I need to make sure I have all the necessary information.\n\nFirst, the title is clearly "Inception". The release year was 2010, I think. Let me confirm that. Yes, Inception came out in 2010. As for the rating, it\'s pretty high. On IMDb, it\'s around 8.8. But I should check if that\'s the total rating they want. Maybe the function expects a number, so I\'ll use that.\n\nWait, the parameters require the rating as a number. IMDb\'s rating is 8.8, so that\'s a number. Okay, that fits. So the arguments would be title: "Inception", year: 2010, rating: 8.8. Let me make sure there\'s no other required fields. The required ones are title, year, and rating. I have all three. \n\nI should structure the tool call correctly. The function name is Movie, an

### NESTED


In [37]:
from pydantic import BaseModel,Field
class Actor(BaseModel):
    name:str
    role:str
class MovieDetails(BaseModel):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]

In [38]:
model_with_structure = model.with_structured_output(MovieDetails)
reponse = model_with_structure.invoke("Provide the details about the Inception Movie")
reponse

MovieDetails(title='Inception', year=2010, cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Ellen Page', role='Ariadne'), Actor(name='Tom Hardy', role='Balthazar')], genres=['Science Fiction', 'Action'])

### Typedict
typedict provide  a simple alertnative using python's built in typinh, ideal when you don't need runtime validation  the major difference between the  pydantic and typedict is about the validation

In [41]:
from typing_extensions import TypedDict,Annotated

class MovieDict(TypedDict):
    """A movie  with details"""
    title:Annotated[str,...,"The title of the movie"]
    year:Annotated[str,...,"The year of the movie"]
mymodel = model.with_structured_output(MovieDict)
myreponse = mymodel.invoke("Give me the description about the Avenger movie")
myreponse

{'title': 'Avenger', 'year': '2012'}

In [42]:
class Actor(TypedDict):
    name:str
    role:str
class YourDetails(TypedDict):
    title:str
    year:int
    cast:list[Actor]
    genres:list[str]
mymodel = model.with_structured_output(YourDetails)
myreponse = mymodel.invoke("Give me the description about the Avenger movie")
myreponse

{'cast': [{'name': 'Robert Downey Jr.', 'role': 'Tony Stark / Iron Man'},
  {'name': 'Chris Evans', 'role': 'Steve Rogers / Captain America'},
  {'name': 'Mark Ruffalo', 'role': 'Bruce Banner / Hulk'},
  {'name': 'Chris Hemsworth', 'role': 'Thor'},
  {'name': 'Scarlett Johansson', 'role': 'Natasha Romanoff / Black Widow'},
  {'name': 'Jeremy Renner', 'role': 'Clint Barton / Hawkeye'}],
 'genres': ['Action', 'Science Fiction', 'Adventure'],
 'title': 'Avengers',
 'year': 2012}

In [43]:
model.profile

{'max_input_tokens': 131072,
 'max_output_tokens': 16384,
 'image_inputs': False,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True}

### Now With Agent

In [45]:
import os
os.environ["GROQ_API_KEY"] = os.getenv("GROK_API_KEY")

In [47]:
from pydantic import BaseModel,Field
from langchain.agents import create_agent


class Contact(BaseModel):
    """extract the contact information"""
    name:str = Field(description="the name of the person")
    email:str  = Field(description="The email address of the person")
    phone:str =Field(description="The phone number of the person")

agent = create_agent(model="groq:qwen/qwen3-32b",response_format=Contact)

result =agent.invoke({"messages":[{"role":"user","content":"Extract the contact information from : Jhon Doe, jhon@example.com,(4455-453-2455)"}]})
result

{'messages': [HumanMessage(content='Extract the contact information from : Jhon Doe, jhon@example.com,(4455-453-2455)', additional_kwargs={}, response_metadata={}, id='1b34f257-12d2-4841-800c-9b676a582d6c'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, let me see. The user wants me to extract contact information from the given text: "Jhon Doe, jhon@example.com,(4455-453-2455)". \n\nFirst, I need to identify the name, email, and phone number. The name is "Jhon Doe". Wait, maybe there\'s a typo here. The first name is spelled "Jhon" instead of "John"? But I should go with what\'s provided.\n\nNext, the email is "jhon@example.com". That looks valid. Then the phone number is "(4455-453-2455)". Hmm, the format is a bit unusual. Normally, phone numbers might have formats like (123) 456-7890 or 123-456-7890. Here it\'s written as 4455-453-2455 with hyphens but without parentheses around the area code. But the user provided it as (4455-453-2455), including the parenth

In [48]:
#check the only structure response
result["structured_response"]

Contact(name='Jhon Doe', email='jhon@example.com', phone='(4455-453-2455)')

### Now with typedict

In [56]:
from typing_extensions import TypedDict,Annotated
from langchain.agents import create_agent
os.environ["GROQ_API_KEY"]=os.getenv("GROK_API_KEY")
class contactInfo(TypedDict):
    """contact info"""
    name:str #the name of the person
    email:str #the email of the person
    phone:str #the phone of the person

agent = create_agent(model="groq:qwen/qwen3-32b",response_format=contactInfo
)

result =agent.invoke({"messages":[{"role":"user","content":"Extract the contact information from : Jhon Doe, jhon@example.com,(4455-453-2455)"}]})
result["structured_response"]

{'name': 'Jhon Doe', 'email': 'jhon@example.com', 'phone': '(4455-453-2455)'}

### Now with dataclasses

In [59]:
from dataclasses import dataclass
from langchain.agents import create_agent
os.environ["GROQ_API_KEY"] = os.getenv("GROK_API_KEY")
@dataclass
class Contactdata:
    """contact should be extracted """
    name:str #name of the contact
    email:str #email of the contact
    phone: str #phone of the contact

agent = create_agent(model="groq:qwen/qwen3-32b",response_format=Contactdata)
reponse = agent.invoke({"messages":[{"role":"user", "content":"Extract contact information from: jhon@example.com, Jhon Doe, 92939923-193932"}]})
reponse["structured_response"]

Contactdata(name='Jhon Doe', email='jhon@example.com', phone='92939923-193932')